In [1]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

Using backend: pytorch


## load

In [2]:
dataset = 'aminer'
df = pd.read_csv(f'../../../01_process/data/{dataset}/Papers.csv') # 5353904
df.dropna(subset=['p_id', 'year'], inplace=True)
df.fillna({'refs': '', 'a_ids': '', 'v_id': '', 'fids': '', 'abst': ''}, inplace=True)
df

,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
0,Parallel Constraint Logic Programming Language...,1.759779e+08,1800.0,6.0,,"2330314058,2809000863,2592030272,2017128800,26...",186357190.0,"4843,4171,1296,7792,639,3215,4,9085,44,4936",
1,A Clipping Divider,2.080845e+09,1899.0,0.0,,"47020662,1941499373",,"26740,1180,159,4,17339,20494",
2,Dynamic Protection Structures,2.048495e+09,1899.0,0.0,,1785551945,,"62,14770,653,907,4",
3,Experiments in the Recognition of Hand-Printed...,2.092848e+09,1899.0,0.0,,"2641132548,2278151506",,"166,4,13006,29",
4,Half-Tone Perspective Drawings By Computer,2.017194e+09,1899.0,0.0,,"2799957682,2800447923,2800385161,2050022765",,"2971,8729,1198",
...,...,...,...,...,...,...,...,...,...
4107364,eMotion: An SGX extension for migrating enclaves,2.894667e+09,2019.0,0.0,"1569778844,1997269120,2061643296,2143818983,21...","2895227020,2895434726,2282325383,2895648844",12529635.0,"10,13231,76,105,3956,322,4,15,5164,189,319",Abstract Software Guard Extensions (SGX) is a ...
4107365,iRobot-Factory: An intelligent robot factory b...,2.887933e+09,2019.0,0.0,"1986755722,2066065861,2172945145,2184902314,22...","2681624575,2679880958,2885033870,2123979082,33...",186357190.0,"13936,1590,103,683,1325,135,15,452,4,48396",Abstract The Internet of Things (IoT) and Arti...
4107366,iSIRA: Integrated shift–invert residual Arnold...,2.765554e+09,2019.0,0.0,"131619556,1207633162,1486991309,1517622252,164...","2109460337,2311862338,2103154382,2106526481",165473669.0,"17,14205,83,19,1510,543,23182,1731,8933,1378,1...",Abstract The eigenvalue problem of a graph Lap...
4107367,sCAKE: Semantic Connectivity Aware Keyword Ext...,2.897012e+09,2019.0,0.0,"171888312,255556494,1509979276,1525595230,1532...","2895899079,2590701236",192650101.0,"30,29,19,9559,704,1921",Abstract Keyword Extraction is an important ta...


In [3]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 559749
papers = df.drop(drop_id) # 3547589
papers

Drop 559749 rows


,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
559768,$P$^$3$$T+$: A performance estimator for distr...,2.137980e+09,2000.0,11.0,"71295551,1496940124,1559570204,1560642162,1575...","1991808295,2475994315",166774750.0,"1309,4,176,7820,322,182,3957,5996,16196,2613",Developing distributed and parallel programs o...
559769,$T$-law of large numbers for fuzzy numbers,2.396726e+09,2000.0,5.0,,2497970465,158577884.0,"19,7276,9752,17,35981,4928,12055,3169,2457",The notions of a t-norm and fuzzy number are r...
559770,$\diamond$ at Mahlo Cardinals,1.543583e+09,2000.0,4.0,"1972177384,1996153963,2000657285,2007636328,20...",2475361447,177675236.0,"27956,17,13668,351,19,13345",Given a Mahlo cardinal k and regular e such th...
559771,'A bit of a mess?': Training and education for...,1.811092e+09,2000.0,5.0,,"2664263981,2701568053",2764383307.0,"37284,556,16,21297,262,3356,598,3216,31054",Despite assertions that the voluntary sector r...
559772,"'Andreas, Rauber'? Conference pages are over t...",2.149053e+09,2000.0,12.0,"1641006451,1830940407,1947395582,2007436697,20...","2057632188,2144383148",1192664773.0,"230,4350,1,238,109,7479,436,2,268,4,4885,935,3462",With the massive advance of electronic documen...
...,...,...,...,...,...,...,...,...,...
4107364,eMotion: An SGX extension for migrating enclaves,2.894667e+09,2019.0,0.0,"1569778844,1997269120,2061643296,2143818983,21...","2895227020,2895434726,2282325383,2895648844",12529635.0,"10,13231,76,105,3956,322,4,15,5164,189,319",Abstract Software Guard Extensions (SGX) is a ...
4107365,iRobot-Factory: An intelligent robot factory b...,2.887933e+09,2019.0,0.0,"1986755722,2066065861,2172945145,2184902314,22...","2681624575,2679880958,2885033870,2123979082,33...",186357190.0,"13936,1590,103,683,1325,135,15,452,4,48396",Abstract The Internet of Things (IoT) and Arti...
4107366,iSIRA: Integrated shift–invert residual Arnold...,2.765554e+09,2019.0,0.0,"131619556,1207633162,1486991309,1517622252,164...","2109460337,2311862338,2103154382,2106526481",165473669.0,"17,14205,83,19,1510,543,23182,1731,8933,1378,1...",Abstract The eigenvalue problem of a graph Lap...
4107367,sCAKE: Semantic Connectivity Aware Keyword Ext...,2.897012e+09,2019.0,0.0,"171888312,255556494,1509979276,1525595230,1532...","2895899079,2590701236",192650101.0,"30,29,19,9559,704,1921",Abstract Keyword Extraction is an important ta...


## test if it is a academic graph

In [4]:
pid2year = {}
pid2cnt = {}
show_2times = []
early_ref = []
for ii in trange(len(papers)): # 先确定每个paper的出现年份
    pid = str(papers['p_id'].iloc[ii])
    year = int(papers['year'].iloc[ii])
    if pid not in pid2year:
        pid2year[pid] = year
        pid2cnt[pid] = 1
    else:
        pid2cnt[pid] += 1
        show_2times.append(pid)

for ii in trange(len(papers)):
    refs = papers['refs'].iloc[ii]
    year = int(papers['year'].iloc[ii])
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            ref = str(ref)
            
            if ref in pid2year and pid2year[ref]>year: # 若tgt出现了，且它的年份>当前年份，则出错
                early_ref.append(ref)
myout(show_2times, early_ref)

100%|██████████| 3547589/3547589 [01:11<00:00, 49676.45it/s]

show_2times : len=0, []
early_ref : len=0, []


## build graph

In [4]:
dataset = 'aminer'
p_id2emb = pkl.load(open(f'../../../01_process/data_relabel/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=4107338, dict([175977922: [ 0.04506357  0.24057241  0.25292784  0.197896   -0.30111322 -0.35823146
  0.01851055 -0.07468222 -0.7361772   0.4787363   0.12397043  0.17263517
 -0.04293935  0.07691953  0.0627557  -0.0402085   0.20825239 -0.17348239
 -0.46093255  0.3343687  -0.12207935 -0.19470656 -0.62984455 -0.03117487
  0.21003556 -0.13791466 -0.05052959  0.47683963 -0.3647912  -0.24764767
  0.64601815 -0.15424506  0.34901485  0.13950773 -0.15486224  0.13388471
  0.11579148 -0.39330736  0.01651833  0.17254636  0.13415797 -0.02703601
 -0.11894822  0.10018919  0.21046153  0.20491917  0.04222823 -0.31911382
  0.15740389 -0.0064537  -0.38944706  0.03027169  0.12148738 -0.15243512
  0.11275464  0.13894105  0.2910288  -0.2664129   0.04032038  0.08143923
  0.00242634 -0.02569124  0.09042111 -0.08137266  0.43134242  0.39026612
  0.0459724  -0.24260871  0.06337933 -0.0520011   0.23559329  0.08813658
  0.19074288 -0.01955303  0.04625257 -0.02598295  0.1647279   0.09184523
  0.295409

In [5]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = int(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            if ref != '':
                ref = int(ref)
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 3547589/3547589 [03:06<00:00, 19041.80it/s]


feat : shape=torch.Size([3864522, 128])
tensor([[ 0.0785, -0.0209,  0.1696,  ..., -0.0469, -0.1611,  0.0522],
        [ 0.0499, -0.0125,  0.1596,  ..., -0.0545, -0.1519,  0.0617],
        [-0.0544, -0.1329,  0.0028,  ..., -0.2587, -0.3068, -0.1364],
        ...,
        [-0.0431, -0.0069,  0.1206,  ..., -0.1017, -0.1419,  0.1529],
        [-0.0113, -0.0008,  0.1514,  ..., -0.1063, -0.0672,  0.0860],
        [ 0.0341, -0.1113,  0.1186,  ..., -0.0446, -0.0713,  0.0649]])
src : shape=torch.Size([33947536]), tensor([      0,       0,       0,  ..., 3864521, 3864521, 3864521])
tgt : shape=torch.Size([33947536]), tensor([      1,       2,       3,  ..., 1400964, 3155609, 3625867])
rel : shape=torch.Size([33947536]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([33947536]), tensor([2000, 2000, 2000,  ..., 2019, 2019, 2019])


In [42]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=3864522, num_edges=33947536,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [43]:
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [12]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [9]:
cites = {}
print(start_year, end_year)
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

papers.fillna({'refs': ''}, inplace=True)
for i in trange(len(papers)):
    year = int(papers['year'].iloc[i])
    refs = papers['refs'].iloc[i]
    if len(refs)>0:
        rlst = refs.split(',')
        rlst = [int(item) for item in rlst]
        for ref in rlst:
            if ref != '':
                cites[year][ref] += 1

2000 2022


100%|██████████| 3547589/3547589 [01:24<00:00, 41924.67it/s]


In [21]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=21, list([0, 419030, 885894, ..., 31834368, 33912556, 33947536])
ts_vals : shape=(20,), [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019]
ts_infos : shape=(20, 3)
[[    2000        0   419030]
 [    2001   419030   885894]
 [    2002   885894  1435564]
 [    2003  1435564  2110301]
 [    2004  2110301  2954516]
 [    2005  2954516  4005903]
 [    2006  4005903  5244870]
 [    2007  5244870  6689757]
 [    2008  6689757  8261795]
 [    2009  8261795 10080537]
 [    2010 10080537 12043677]
 [    2011 12043677 14225152]
 [    2012 14225152 16604037]
 [    2013 16604037 19266155]
 [    2014 19266155 22193719]
 [    2015 22193719 25260892]
 [    2016 25260892 28519420]
 [    2017 28519420 31834368]
 [    2018 31834368 33912556]
 [    2019 33912556 33947536]]


In [35]:
end_year = 2020
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf
    

2018: 100%|██████████| 1/1 [00:00<00:00, 26.96it/s, pdf=123466, year=2018]
2019: 0it [00:00, ?it/s]


In [36]:
labels[2005]

,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,2093225945,1810,25.0,36.0,45.0,23.0,45.0,20.0,33.0,48.0,36.0,37.0,32.0,25.0,20.0,0.0
1,1969215126,1926,37.0,39.0,42.0,47.0,44.0,42.0,38.0,19.0,20.0,2.0,1.0,1.0,0.0,0.0
2,2138363365,5153,245.0,222.0,251.0,250.0,177.0,200.0,188.0,153.0,133.0,116.0,78.0,48.0,32.0,0.0
3,1644882639,5688,108.0,120.0,112.0,88.0,81.0,53.0,33.0,41.0,45.0,38.0,23.0,26.0,13.0,0.0
4,2402007929,8769,3.0,4.0,4.0,6.0,0.0,1.0,4.0,9.0,7.0,2.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111229,2126284512,868552,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111230,2047154474,868555,0.0,2.0,2.0,1.0,2.0,0.0,0.0,2.0,4.0,1.0,1.0,1.0,0.0,0.0
111231,1556533256,868557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111232,1604375569,868559,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

In [39]:
labels[2018]

,id,nid,2019
0,2498524691,382878,1.0
1,2135668496,478721,0.0
2,2151477234,528121,0.0
3,2497711791,560131,0.0
4,2156539432,599447,0.0
...,...,...,...
123461,2788268216,3862311,0.0
123462,2885827770,3862312,0.0
123463,2617541291,3862324,0.0
123464,2796051876,3862329,0.0


## cum log labels

In [40]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-2): # 2000, 2020
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[2005]

19


,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,2093225945,1810,3.258096,4.127134,4.672829,4.867535,5.164786,5.273000,5.429346,5.620401,5.743003,5.855072,5.942800,6.006353,6.054440,6.054440
1,1969215126,1926,3.637586,4.343805,4.779123,5.111988,5.347107,5.529429,5.669881,5.733341,5.796058,5.802118,5.805135,5.808143,5.808143,5.808143
2,2138363365,5153,5.505332,6.148468,6.577861,6.876265,7.044033,7.204893,7.335634,7.430707,7.506592,7.568379,7.607878,7.631432,7.646832,7.646832
3,1644882639,5688,4.691348,5.433722,5.831882,6.061457,6.234411,6.333280,6.390241,6.456769,6.525030,6.579251,6.610696,6.645091,6.661855,6.661855
4,2402007929,8769,1.386294,2.079442,2.484907,2.890372,2.890372,2.944439,3.135494,3.465736,3.663562,3.713572,3.737670,3.737670,3.761200,3.761200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111229,2126284512,868552,0.693147,1.098612,1.609438,1.791759,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910,1.945910
111230,2047154474,868555,0.000000,1.098612,1.609438,1.791759,2.079442,2.079442,2.079442,2.302585,2.639057,2.708050,2.772589,2.833213,2.833213,2.833213
111231,1556533256,868557,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
111232,1604375569,868559,0.693147,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612,1.098612


In [41]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))